## Extended Kalman Filter
I decided to implement this first in Python instead of just filling in the blanks and patching together the boilerplate code from Udacity.

This will allow me to code it up from scratch and really get an intimate understanding of Kalman Filters before I complete the implementation in C++.

Using a Jupyter notebook will also help me to take thorough notes on the math in the midst of the code.

In EKF.py, I recreate the normal Kalman Filter implementation that I already built in C++ during the lessons.  Then I filled in the gaps in the implementation.

In [1]:
from EKF import *

In [2]:
#//input file with laser and radar measurements
in_file_name_ = "../data/sample-laser-radar-measurement-data-1.txt"  

#//output file with the estimation (px, py, vx, vy), measurements (px and py meas) and ground truths (px, py, vx, vy)
out_file_name_ = "obj_pose-laser-radar-output.txt"

In [3]:
print "RMSE:",run_ekf(in_file_name_, (.163 , .000041))

RMSE: [[ 0.02448499]
 [ 0.06257636]
 [ 0.03133589]
 [ 0.01819227]]



# Discern an optimal set of noise parameters

In [4]:
import itertools
import glob

def meets_specifications(rmse):
    """
    Takes in both RMSE values
    Returns True if the RMSE is within specified values
    """
    first_rmse = rmse[0]
    second_rmse = rmse[1]
    if first_rmse[0] <= .07 and \
        first_rmse[1] <= .07 and \
        first_rmse[2] <= .6 and \
        first_rmse[3] <= .6 and \
        second_rmse[0] <= .2 and \
        second_rmse[1] <= .2 and \
        second_rmse[2] <= .5 and \
        second_rmse[3] <= .83:
        return True 

In [13]:
# Initialize a high value for the total mean squared error (Between both files)
best_total = 100
pro_noise_range = np.arange(0.161,.162,.001)
meas_noise_range = np.arange(0.0000405,0.0000415,.0000001)
i = 0
for pro_noise, meas_noise in itertools.product(pro_noise_range, meas_noise_range):
    i += 1
    total = 0
    rmse = [None,None]
    for j, in_file_name_ in enumerate(glob.glob("../data/*")):
        rmse[j] = run_ekf(in_file_name_, (pro_noise, meas_noise))
        total += np.sum(rmse[j])
    print str(i) + ":", pro_noise, meas_noise, total, "Best:", best_total
    if best_total > total and meets_specifications(rmse):
        best_total = total
        best_values = pro_noise, meas_noise, rmse

1: 0.161 4.05e-05 0.291394688643 Best: 100
2: 0.161 4.06e-05 0.284914450108 Best: 0.291394688643
3: 0.161 4.07e-05 0.285211716422 Best: 0.284914450108
4: 0.161 4.08e-05 0.289463715333 Best: 0.284914450108
5: 0.161 4.09e-05 0.295346788801 Best: 0.284914450108
6: 0.161 4.1e-05 0.302170153386 Best: 0.284914450108
7: 0.161 4.11e-05 0.309937937755 Best: 0.284914450108
8: 0.161 4.12e-05 0.318994857991 Best: 0.284914450108
9: 0.161 4.13e-05 0.329641641762 Best: 0.284914450108
10: 0.161 4.14e-05 0.341677946327 Best: 0.284914450108
11: 0.162 4.05e-05 0.322809922631 Best: 0.284914450108
12: 0.162 4.06e-05 0.309218263532 Best: 0.284914450108
13: 0.162 4.07e-05 0.296807595009 Best: 0.284914450108
14: 0.162 4.08e-05 0.287437468756 Best: 0.284914450108
15: 0.162 4.09e-05 0.284325156555 Best: 0.284914450108
16: 0.162 4.1e-05 0.286909345677 Best: 0.284325156555
17: 0.162 4.11e-05 0.292049871514 Best: 0.284325156555
18: 0.162 4.12e-05 0.298357329762 Best: 0.284325156555
19: 0.162 4.13e-05 0.30553805403

In [14]:
print best_values

(0.16200000000000001, 4.0900000000000012e-05, [matrix([[ 0.0245491 ],
        [ 0.06262924],
        [ 0.02555261],
        [ 0.01661092]]), matrix([[ 0.02132479],
        [ 0.01438511],
        [ 0.02233536],
        [ 0.09693804]])])


From the above, I figure that a good measurment noise is 0.0000409, and a good process noise is 0.162.

I found these value by finding optimal values for a wide range, then narrowing my each around optimal values further and searching again.  I did that iteration only a few times.